In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from datetime import datetime
from pytz import timezone

src = '****'
local = '*****'
val1 = datetime.strptime('2020-03-02 15:52:38.000000','%Y-%m-%d %H:%M:%S.%f')

In [2]:
path = '**\\WiresharkCapture_20000.csv'

In [3]:
def getFileName(path):
    return getCase(path)+"_"+os.path.basename(path)

In [4]:
def getCase(path):
    #print(path)
    case = ""
    if("2005" in path):
        case = "RM11"
    if("2006" in path):
        #print("inside if")
        case = "RM22"
    if("2001" in path):
        case = "PV33"
    if("2000" in path):
        case = "meterr"
    #print("case : "+case)
    return case

In [5]:
def readCsv(path):
    df = pd.read_csv(path)
    return df

In [6]:
def appendToFrame(df,src,local,path,val1): 
    df['frame.time']=df['frame.time'].str[:-25]
    df['frame.time'] = pd.to_datetime(df['frame.time'])
    df = df.sort_values(by=['frame.time'], ascending=True)
    df['time_local'] = df['frame.time'].dt.tz_localize('EST').dt.tz_convert('UTC')
    df['time_local']= df['time_local'].apply(lambda d: d.replace(tzinfo=None))
    df = df.sort_values(by=['time_local'], ascending=True)
    df['delta'] = df['time_local'].apply(lambda d: d - val1)
    df['delta'] = df['delta'].apply(lambda d: (str(d.days)+" days, "+str(round(d.seconds//3600))+" hours"))
    inbound=0
    outbound=0
    scan =0
    control = 0
    scan_rate=0
    ar=[]
    temp_date = df['delta'].iloc[0]
    ar.append(['DER_case','time_UTC','Frequency Of Scans','Frequency of Control Signals','Inbound','Outbound','Scan Rate'])
    for row in range(len(df)):
        #df['frame.time'][row]=timezone('EST').localize(df['frame.time'][row])
        #df['delta'].iloc[row] = df['frame.time'].iloc[row] - val1
        #df['delta'].iloc[row] = str(df['delta'].iloc[row].days)+" days, "+str(round(df['delta'].iloc[row].seconds/3600))+" hour"
        #print(df['delta'].iloc[row])
        #if(row==0):
        #df = df.sort_values(by=['frame.time'], ascending=True)
        #print(temp_date)
        if(df.iloc[row]['delta']==temp_date):
            if(df.iloc[row]['ip.src']==src and df.iloc[row]['ip.dst']==local):
                inbound = df.iloc[row]['frame.len']+inbound
            if(df.iloc[row]['ip.src']==local and df.iloc[row]['ip.dst']==src):
                outbound = df.iloc[row]['frame.len']+outbound
            if("Read" in df.iloc[row]['_ws.col.Info']):
                scan = scan+1
            if("Direct Operate" in df.iloc[row]['_ws.col.Info']):
                control = control+1
        if(df.iloc[row]['delta']!=temp_date):
            if(temp_date.startswith('0 days')):
                ar.append([getCase(path),temp_date,scan,control,inbound,outbound,(scan/60)])
            temp_date = df.iloc[row]['delta']
            scan = 0
            control = 0
            inbound = 0
            outbound = 0
            if(df.iloc[row]['ip.src']==src and df.iloc[row]['ip.dst']==local):
                inbound = df.iloc[row]['frame.len']+inbound
            if(df.iloc[row]['ip.src']==local and df.iloc[row]['ip.dst']==src):
                outbound = df.iloc[row]['frame.len']+outbound
            if("Read" in df.iloc[row]['_ws.col.Info']):
                scan = scan+1
            if("Direct Operate" in df.iloc[row]['_ws.col.Info']):
                control = control+1
    if(temp_date.startswith('0 days')):
        ar.append([getCase(path),temp_date,scan,control,inbound,outbound,(scan/60)])
    saveFrameToCSV(ar,path)
    print("File Saved Successfully")

In [7]:
def saveFrameToCSV(ar,path):
    cs = pd.DataFrame(data=ar[1:], columns=ar[0])
    #print(cs.describe())
    #cs.index = cs['DER_case']
    cs = cs.iloc[1:]
    #cs.drop(cs.iloc[0],inplace=True)
    cs.to_csv(getFileName(path),index=False, index_label=False)
    cs['sum'] = cs['Inbound']+cs['Outbound']
    print("Peak Hour")
    print(cs[cs['sum']==cs['sum'].max()].iloc[0])
    print("---------------------")
    print("Non-peak Hour")
    print(cs[cs['sum']<=round(cs['sum'].mean())].iloc[0])
    #if(cs['Frequency of Control Signals'].max()==0):
     #   cs1 = cs[cs['Frequency Of Scans'] ==cs['Frequency Of Scans'].max()]
     #   print(cs1[cs1['Inbound']==cs1['Inbound'].max()].iloc[0])
   # else:
    #    print(cs[cs['Frequency of Control Signals']==cs['Frequency of Control Signals'].max()].iloc[0])
   # print(cs[cs['Frequency Of Scans']==cs['Frequency Of Scans'].min()].iloc[0])
    return cs